In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
from tifffile import imread
from wordcloud import WordCloud

In [ ]:
import keras

In [ ]:
import os

In [ ]:
import keras, os

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, BatchNormalization, Activation, LSTM, CuDNNLSTM, Embedding, Dropout
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from tifffile import imread
from wordcloud import WordCloud

import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, BatchNormalization, Activation, LSTM, CuDNNLSTM, Embedding, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.optimizers import Adam

In [ ]:
df_train = pd.read_csv('./new_data/train_accident_normalized_text.csv')
df_test = pd.read_csv('./new_data/test_accident_normalized_text.csv')

In [ ]:
label_count = df_train['target'].value_counts()
labels = ['Negative', 'Positive']

plt.pie(label_count, labels=labels, autopct='%1.1f%%')

plt.show()

print(label_count[0] / label_count[1])